In [ ]:
import json
import matplotlib.pyplot as plt
import os
import geopandas as gpd
from shapely.geometry import shape, Point
from matplotlib.collections import LineCollection
import svgutils.transform as sg  # Import svgutils for SVG manipulation

# Load data and create GeoDataFrames
with open('./Berlin.json', 'r') as file:
    berlin_data = json.load(file)
with open('./centroids.json', 'r') as file:
    centroids_data = json.load(file)

# Define color and line width
COLOR = "#1E3791"
LINE_WIDTH = 13

# Path to the additional SVG file to embed
additional_svg_path = './mini-uebersichtskarte.svg'

# Convert Berlin outline to a GeoDataFrame
berlin_features = [shape(feature['geometry']) for feature in berlin_data['features']]
gdf_berlin = gpd.GeoDataFrame(geometry=berlin_features, crs="EPSG:4326")  # Assuming WGS84

# Convert centroids to a GeoDataFrame, extracting "pgr_name" for each centroid
centroid_points = []
pgr_names = []
for feature in centroids_data['features']:
    centroid_points.append(Point(feature['geometry']['coordinates']))
    pgr_names.append(feature['properties'].get("pgr_name", "Unnamed"))

gdf_centroids = gpd.GeoDataFrame(geometry=centroid_points, crs="EPSG:4326")
gdf_centroids["pgr_name"] = pgr_names

# Reproject both GeoDataFrames to pseudo-Mercator (EPSG:3857)
gdf_berlin = gdf_berlin.to_crs("EPSG:3857")
gdf_centroids = gdf_centroids.to_crs("EPSG:3857")

# Calculate the aspect ratio of the Berlin bounding box
bounds = gdf_berlin.total_bounds
width_range = bounds[2] - bounds[0]
height_range = bounds[3] - bounds[1]
aspect_ratio = height_range / width_range

# Define SVG width and calculate height based on the aspect ratio
svg_width = 800  # Width in pixels
svg_height = int(svg_width * aspect_ratio)  # Calculate height to maintain aspect ratio

# Define padding as 5% of the width and height
padding_x = width_range * 0.05
padding_y = height_range * 0.05

# Create output directory if it doesn't exist
output_folder = "./svgs"
os.makedirs(output_folder, exist_ok=True)

# Iterate through each centroid feature and create an SVG
for _, row in gdf_centroids.iterrows():
    centroid = row.geometry
    pgr_name = row["pgr_name"].replace(" ", "_").replace("/", "_").replace(";", "_")

    # Create a new plot with the calculated aspect ratio
    fig, ax = plt.subplots(figsize=(svg_width / 100, svg_height / 100), dpi=100)
    ax.set_aspect('equal')
    
    # Plot the Berlin outline using LineCollection with rounded line caps
    # for geom in gdf_berlin.geometry:
    #     if geom.geom_type == 'Polygon':
    #         coords = geom.exterior.coords
    #         line = LineCollection([coords], color=COLOR, linewidth=1, capstyle='round')
    #         ax.add_collection(line)
    #     elif geom.geom_type == 'MultiPolygon':
    #         for poly in geom:
    #             coords = poly.exterior.coords
    #             line = LineCollection([coords], color=COLOR, linewidth=1, capstyle='round')
    #             ax.add_collection(line)

    # Plot the centroid point
    ax.plot(centroid.x, centroid.y, color=COLOR, marker='o', markersize=13)
    
    # Set limits with calculated padding and remove axes for cleaner output
    ax.set_xlim(bounds[0] - padding_x, bounds[2] + padding_x)
    ax.set_ylim(bounds[1] - padding_y, bounds[3] + padding_y)
    ax.axis('off')  # Turn off axis
    
    # Save the SVG with pgr_name in the filename
    base_svg_filename = os.path.join(output_folder, f'{pgr_name}_base.svg')
    plt.savefig(base_svg_filename, format='svg', bbox_inches='tight', pad_inches=0)
    plt.close(fig)  # Close the figure to free memory

    # Use svgutils to embed an additional SVG
    base_svg = sg.fromfile(base_svg_filename)
    additional_svg = sg.fromfile(additional_svg_path)
    
    # Adjust the size and position of the additional SVG (e.g., scale and translate as needed)
    additional_svg_root = additional_svg.getroot()
    additional_svg_root.scale(0.7)  # Scale down the additional SVG to 50%
    additional_svg_root.moveto(20, 20)  # Move it to (x, y) position

    # Append the additional SVG to the base SVG
    base_svg.append(additional_svg_root)
    
    # Save the combined SVG
    combined_svg_filename = os.path.join(output_folder, f'{pgr_name}.svg')
    base_svg.save(combined_svg_filename)
    
    print(f"Saved {combined_svg_filename}")


Saved ./svgs/Südlicher_Prenzlauer_Berg.svg
Saved ./svgs/Kreuzberg_Süd.svg
Saved ./svgs/Marienfelde___Lichtenrade.svg
Saved ./svgs/Wilhelmstadt___Staaken.svg
Saved ./svgs/Nördliches_Weißensee.svg
Saved ./svgs/Tempelhof.svg
Saved ./svgs/Lichtenberg_Nord.svg
Saved ./svgs/Lichtenberg_Mitte.svg
Saved ./svgs/Hellersdorf.svg
Saved ./svgs/Buch.svg
Saved ./svgs/Hohenschönhausen_Nord.svg
Saved ./svgs/Kaulsdorf_Mahlsdorf.svg
Saved ./svgs/Treptow-Köpenick_1.svg
Saved ./svgs/Tegel.svg
Saved ./svgs/Mariendorf.svg
Saved ./svgs/Friedrichshain_Ost.svg
Saved ./svgs/Nördlicher_Prenzlauer_Berg.svg
Saved ./svgs/Charlottenburg_Nord.svg
Saved ./svgs/Lankwitz_Lichterfelde_Ost.svg
Saved ./svgs/Schöneberg_Nord.svg
Saved ./svgs/Neukölln.svg
Saved ./svgs/Zehlendorf_Nord_Wannsee.svg
Saved ./svgs/Wilmersdorf_Süd.svg
Saved ./svgs/Grüner_Norden.svg
Saved ./svgs/Zentrum.svg
Saved ./svgs/Schöneberg_Süd.svg
Saved ./svgs/Gesundbrunnen.svg
Saved ./svgs/Gatow___Kladow.svg
Saved ./svgs/Nördliches_Pankow.svg
Saved ./svgs/Tre